In [2]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

mnist = load_digits()
x,test_x,y,test_y = train_test_split(mnist.data,mnist.target,test_size=0.25,random_state=40)

In [16]:
class MultinomialNaiveBayes:
    def __init__(self):
        self.class_log_prior_ = None
        self.feature_log_prob_ = None

    def fit(self, X, y):
        # 计算每个类的先验概率和保存为log值
        num_classes = len(np.unique(y))
        self.class_log_prior_ = np.log(np.bincount(y) / len(y))
        
        # 初始化存储特征条件概率的数组
        self.feature_log_prob_ = np.zeros((num_classes, X.shape[1]))
        
        # 计算每个类中每个特征的条件概率
        for c in np.unique(y):
            X_c = X[y == c]
            self.feature_log_prob_[c, :] = np.log((X_c.sum(axis=0) + 88) / (np.sum(X_c.sum(axis=0) -111)))

    def predict(self, X):
        # 利用Bayes公式计算每个样本属于每个类的概率
        log_prob = self.class_log_prior_ + X @ self.feature_log_prob_.T
        return np.argmax(log_prob, axis=1)

# 创建并训练模型
model = MultinomialNaiveBayes()
model.fit(x, y)

# 进行预测
z = model.predict(test_x)

# 计算准确率
print('准确率：', np.sum(z == test_y) / len(test_y))

准确率： 0.8711111111111111


1. **先验概率的计算**

    公式：$ P(y) $
    ```python
    self.class_log_prior_ = np.log(np.bincount(y) / len(y))
    ```
    这一行计算每个类$ y $的先验概率，并将结果保存为对数值。`np.bincount(y)`计算每个类在数据集中出现的次数，然后除以总样本数得到每个类的先验概率。

2. **计算特征的条件概率**

    公式：$ P(x_i | y) = \frac{{\text{Count}(x_i, y) + 1}}{{\text{Count}(y) + |V|}} $
    ```python
    for c in np.unique(y):
        X_c = x[y == c]
        self.feature_log_prob_[c, :] = np.log((X_c.sum(axis=0) + 1) / (np.sum(X_c.sum(axis=0) + 1)))
    ```
    这一部分在循环中对每个类$ c $分别计算所有特征$ x_i $的条件概率。注意，这里使用了Laplace平滑（加1平滑）。

### 预测

1. **计算后验概率**

    公式：$\hat{y} = \arg\max_y \left[ \log P(y) + \sum_{i=1}^{n} \log P(x_i | y) \right]$
    ```python
    log_prob = self.class_log_prior_ + X @ self.feature_log_prob_.T
    ```
    这一行使用先验概率的对数（`self.class_log_prior_`）和特征条件概率的对数（`self.feature_log_prob_`）来计算后验概率的对数。这里使用矩阵乘法简化了求和操作。

2. **选择最高后验概率的类作为预测**

    ```python
    return np.argmax(log_prob, axis=1)
    ```
    这里我们简单地选择具有最高后验概率的类作为每个样本的预测标签。

新增超参：拉普拉斯平滑

